In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, argparse, csv
import shutil
from pprint import pprint

In [3]:
import _mypath
from bripipetools.globusgalaxy import curation

In [4]:
batch_file = '/Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/globus_batch_submission/160412_P43-12_C6VG0ANXX_optimized_nextera_sr_grch38_v0.1_complete.txt'
batch_file2 = '/Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/globus_batch_submission/160216_P43-12_C6VG0ANXX_optimized_nextera_sr_grch38_v0.1_complete.txt'

In [119]:
bc = curation.BatchCurator(batch_file)

In [120]:
bc.organize_files(dry_run=True)


Workflow: optimized_nextera_sr_grch38_v0.1_complete
>> Compiling ouputs for lib6839_C6VG0ANXX [P43-12] (1 of 1)

 > Result source: fastqc
   - Creating directory /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX/qcR1
   - Creating directory /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX/qcR1
   (file 1 of 2)
   - Copying /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX_fastqc_qc.txt to /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX/qcR1/fastqc_data.txt
   (file 2 of 2)
   - Copying /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX_fastqc_qc.html to /Volumes/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Project_P43-12Processed_160412/QC/lib6839_C6VG0ANXX/qcR1/fastqc_report.html
   - Zipp